# Imports

In [2]:
import torch

In [31]:
training_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [1]:
import pandas as pd
df=pd.read_csv("Data/cf_train.csv")

# Custom Dataloader

In [32]:
class CustomDataset:
    def __init__(self,dataframe,batch_size,device =training_device,shuffle=False):
        self.df=dataframe
        self.batch_size=batch_size
        self.columns_to_drop=['row_num','day','era','target_10_val','target_5_val','sigma','day_no']
        self.X = self.df.drop(self.columns_to_drop, axis=1)
        self.y=self.df['target_10_val']
        self.device=device
        self.shuffle=shuffle

    def generate_batches_with_labels(self,idx):
        data=self.X.iloc[:max(0,idx-10)]
        labels=self.y.iloc[:max(0,idx-10)]
        dataset =  torch.utils.data.TensorDataset(torch.tensor(data.values),torch.tensor(labels.values))
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, shuffle=self.shuffle)
        data_unseen = self.X.iloc[max(0,idx-9):idx+1]
        labels_unseen=self.y.iloc[max(0,idx-9):idx+1]
        data_unseen,labels_unseen = torch.tensor(data_unseen.values).to(self.device),torch.tensor(labels_unseen.values).to(self.device)
        return dataloader, (data_unseen,labels_unseen)
    
        
        

In [35]:
customDS = CustomDataset(df,64,shuffle = False)
supervised_dl, unsupervised_data = customDS.generate_batches_with_labels(77)